# Detekcia tváre
---
DP2020
---

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import dlib
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from pathlib import Path
from helpers.pickleHelper import PickleHelper

In [ ]:
# GPU present test
with tf.Session() as sess:
    devices = sess.list_devices()
    print(devices)

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## DLib
---

In [ ]:
import dlib
import shutil

In [ ]:
# Path to folder with bw images not segmentated - only
path_not_segmentated_bw_only = Path("data/final_output/bw_not_segmentated/imgs")
# Path to folder with bw images
path_bw = Path("data/final_output/bw/imgs")
# Path to folder with colorized images
path_rgb = Path("data/final_output/rgb/imgs")

In [ ]:
get_bw_img = lambda x: path_bw / f"{x.name}"

In [ ]:
detector = dlib.get_frontal_face_detector()

In [ ]:
data = {}

for n, bw in enumerate(path_not_segmentated_bw_only.glob("*.jpg")):
    print(f"Processed image n.{n}")

#     bw = get_bw_img(rgb)

    image_np_rgb = np.array(Image.open(rgb))
    image_np_bw = np.array(Image.open(bw))
    if len(image_np_bw.shape) < 3 or image_np_bw.shape[2] == 1:
        image_np_bw = np.dstack([image_np_bw, image_np_bw, image_np_bw])
    
#     print(f"{rgb}: {image_np_rgb.shape}")
#     print(f"{bw}: {image_np_bw.shape}")

    dets_rgb, scores_rgb, idx = detector.run(image_np_rgb, 1)
    dets_bw, scores_bw, idx = detector.run(image_np_bw, 1)
    data[bw.name] = {"bw": [dets_bw, scores_bw], "rgb": [dets_rgb, scores_rgb]}
    
PickleHelper.save("data/final_output/pkls/not_segmentated/FACE.pkl", data)   

In [ ]:
data = PickleHelper.load("data/final_output/pkls/not_segmentated/FACE.pkl"); len(data.keys())

## RetinaFace
---

In [ ]:
import cv2

import numpy as np
import datetime
import os
import glob
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

from pathlib import Path
from insightface.RetinaFace.retinaface import RetinaFace
from helpers.pickleHelper import PickleHelper

In [ ]:
thresh = 0.01
scales = [1024, 1980]

count = 1

gpuid = 0
detector = RetinaFace('insightface/RetinaFace/models/retinaface/R50', 0, gpuid, 'net3')

### Process analyse from folder of images and create .pkl

In [ ]:
# Path to folder with bw images
path_bw = Path("data/final_output/bw/imgs")
# Path to folder with colorized images
path_rgb = Path("data/final_output/rgb/imgs")

In [ ]:
def save_image(img, name, ext):
    filename = f'./{name}{ext}'
    print('writing', filename)
    cv2.imwrite(filename, img)

In [ ]:
def show_image(frame: np.ndarray):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    plt.figure()
    plt.imshow(frame.astype(np.uint8))
    plt.axis('off')
    plt.show()

In [ ]:
def print_result(img, faces, landmarks, printout):
    if faces is not None:
        print(f'{printout} find', faces.shape[0], 'faces')
        for i in range(faces.shape[0]):
            print(f'{printout} score', faces[i][4])
            box = faces[i].astype(np.int)
            print(f'{printout} box', box)
            #color = (255,0,0)
            color = (0,0,255)
            cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, 2)
            if landmarks is not None:
                landmark5 = landmarks[i].astype(np.int)
                #print(landmark.shape)
                for l in range(landmark5.shape[0]):
                    color = (0,0,255)
                    if l==0 or l==3:
                        color = (0,255,0)
                    cv2.circle(img, (landmark5[l][0], landmark5[l][1]), 1, color, 2)
                
    return img

In [ ]:
get_bw_img = lambda x: path_bw / f"{x.name}"

In [ ]:
save_result = False
show_result = False

data = {}
for n, rgb in enumerate(path_rgb.glob("*.jpg")):
    print(f"Processed image n.{n} - {rgb}")
    img_name = rgb.name
    
    bw_path = get_bw_img(rgb)
    rgb_path = rgb

    
    bw = cv2.imread(bw_path.__str__())
    rgb = cv2.imread(rgb_path.__str__())

    if len(bw.shape) < 3 or bw.shape[2] == 1:
        bw = np.dstack([bw, bw, bw])
    
    bw_im_shape = bw.shape
    rgb_im_shape = rgb.shape
    
    target_size = scales[0]
    max_size = scales[1]

    bw_im_size_min = np.min(bw_im_shape[0:2])
    bw_im_size_max = np.max(bw_im_shape[0:2])
    
    rgb_im_size_min = np.min(rgb_im_shape[0:2])
    rgb_im_size_max = np.max(rgb_im_shape[0:2])
    
    bw_im_scale = float(target_size) / float(bw_im_size_min); bw_im_scale
    rgb_im_scale = float(target_size) / float(rgb_im_size_min); rgb_im_scale

    if np.round(bw_im_scale * bw_im_size_max) > max_size:
        bw_im_scale = float(max_size) / float(bw_im_size_max); bw_im_scale
    bw_scales = [bw_im_scale]
    
    if np.round(rgb_im_scale * rgb_im_size_max) > max_size:
        rgb_im_scale = float(max_size) / float(rgb_im_size_max); rgb_im_scale
    rgb_scales = [rgb_im_scale]
    
    
    bw_faces, bw_landmarks = detector.detect(bw, thresh, scales=bw_scales, do_flip=False)
    rgb_faces, rgb_landmarks = detector.detect(rgb, thresh, scales=rgb_scales, do_flip=False)

    if save_result or show_result:
        bw_res = print_result(bw, bw_faces, bw_landmarks, "[BW]")
        rgb_res = print_result(rgb, rgb_faces, rgb_landmarks, "[RGB]")
    
    if save_result:
        save_image(rgb_res, f"{rgb_path.stem}_rgb_o", rgb_path.suffix)
        save_image(bw_res, f"{bw_path.stem}_bw_o", bw_path.suffix)
        
    if show_result:
        show_image(rgb_res)
        show_image(bw_res)
        
    data[img_name] = {"rgb": rgb_faces, "bw": bw_faces}
    
PickleHelper.save("data/final_output/pkls/segmentated_rgb_bw/RETINATNET_001.pkl", data)   

## ImageNet
---

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
import time
from PIL import Image
from nets import inception_resnet_v2
from preprocessing import inception_preprocessing
from helpers.pickleHelper import PickleHelper

from datasets import dataset_utils
from pathlib import Path

# Main slim library
from tensorflow.contrib import slim

In [ ]:
# Path to folder with bw images
path_bw = Path("data/detection_task/bw_in_orig_size")
# Path to folder with colorized images
path_rgb = Path("data/detection_task/rgb")

In [ ]:
get_bw_img = lambda x: path_bw / f"{x.name}"

In [ ]:
url = "http://download.tensorflow.org/models/inception_resnet_v2_2016_08_30.tar.gz"

if not tf.gfile.Exists(checkpoints_dir):
    tf.gfile.MakeDirs(checkpoints_dir)

    dataset_utils.download_and_uncompress_tarball(url, checkpoints_dir)

In [ ]:
def shw_image(frame: np.ndarray):
        plt.figure()
        plt.imshow(frame.astype(np.uint8))
        plt.axis('off')
        plt.show()

In [ ]:
def detect(path_to_model, path_to_image):
    image_size = inception_resnet_v2.inception_resnet_v2.default_image_size

    with tf.Graph().as_default():
        frame = open(path_to_image, 'rb').read()
        image = tf.image.decode_jpeg(frame, channels=3)
        processed_image = inception_preprocessing.preprocess_image(image, image_size, image_size, is_training=False)
        processed_images  = tf.expand_dims(processed_image, 0)
    
        # Create the model, use the default arg scope to configure the batch norm parameters.
        with slim.arg_scope(inception.inception_resnet_v2_arg_scope()):
            logits, _ = inception_resnet_v2.inception_resnet_v2(processed_images, num_classes=1001, is_training=False)
        probabilities = tf.nn.softmax(logits)
        
        init_fn = slim.assign_from_checkpoint_fn(
            os.path.join(checkpoints_dir, path_to_model),
            slim.get_model_variables('InceptionResnetV2'))
        
        with tf.Session() as sess:
            init_fn(sess)
            np_image, probabilities = sess.run([image, probabilities])
            probabilities = probabilities[0, 0:]
            sorted_inds = [i[0] for i in sorted(enumerate(-probabilities), key=lambda x:x[1])]
        
#         plt.figure()
#         plt.imshow(np_image.astype(np.uint8))
#         plt.axis('off')
#         plt.show()
        
        return probabilities, sorted_inds
        
#         names = imagenet.create_readable_names_for_imagenet_labels()
#         for i in range(5):
#             index = sorted_inds[i]
#             print('Probability %0.2f%% => [%s]' % (probabilities[index] * 100, names[index]))
        

## MTCNN
---

In [ ]:
from mtcnn import MTCNN
import cv2
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from pathlib import Path
import dlib
from utils.pickleHelper import PickleHelper

# Path to folder with bw images
path_bw = Path("data/detection_task/bw_in_orig_size")
# Path to folder with colorized images
path_rgb = Path("data/detection_task/rgb")
# Path to ground-truth .pkl file
path_gt = PickleHelper.load("data/final_output/pkls/segmentated_rgb_bw/gt_250_faces.pkl")

get_bw_img = lambda x: path_bw / f"{x}"
get_rgb_img = lambda x: path_rgb / f"{x}"

detector = MTCNN()

In [ ]:
data = {}
for n, img_name in enumerate(path_gt):
#     img_name = "Commonwealth Bank staff ca. 1925.jpg"
    print(f"Processed image n.{n} - {img_name}")

    bw_path = get_bw_img(img_name)
    rgb_path = get_rgb_img(img_name)

    bw = cv2.cvtColor(cv2.imread(bw_path.__str__()), cv2.COLOR_BGR2RGB)
    rgb = cv2.cvtColor(cv2.imread(rgb_path.__str__()), cv2.COLOR_BGR2RGB)
    
    if len(bw.shape) < 3 or bw.shape[2] == 1:
        bw = np.dstack([bw, bw, bw])
    
#     shw_image(bw)
#     shw_image(rgb)
    
    dets_bw = detector.detect_faces(bw)
    dets_rgb = detector.detect_faces(rgb)
    
#     print(len(dets_bw))
#     print(len(dets_rgb))
    rgb_rects = list()
    rgb_scores = list()
    for d in dets_rgb:
        # Bbox
        rgb_rects.append(dlib.rectangle(left=d['box'][0], top=d['box'][1], right=d['box'][0] + d['box'][2], bottom=d['box'][1] + d['box'][3]))

        # Score
        rgb_scores.append(d['confidence']) 
    
    
    bw_rects = list()
    bw_scores = list()
    for d in dets_bw:
        # Bbox
        bw_rects.append(dlib.rectangle(left=d['box'][0], top=d['box'][1], right=d['box'][0] + d['box'][2], bottom=d['box'][1] + d['box'][3]))

        # Score
        bw_scores.append(d['confidence'])
    
    data[img_name] = {"rgb": [rgb_rects, rgb_scores], "bw": [bw_rects, bw_scores]}

PickleHelper.save("data/final_output/pkls/segmentated_rgb_bw/MTCNN.pkl", data)   